# Initialisation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import yfinance as yf
from google.colab import drive
import gspread

import os
import pandas as pd
import pandas_datareader as pdr
import datetime
import csv
import pytz
import json

import sqlite3
import numpy as np
import concurrent.futures

import requests
from bs4 import BeautifulSoup

database_path = "/content/drive/MyDrive/Colab_Notebook/Data/SP500/stocks.db"
table_name = "stocks"

# Table Creation

In [ ]:
# Create options historical data table
def create_options_table(database_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Create table query
    create_table_query = '''
    CREATE TABLE IF NOT EXISTS historical_options (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        contractID TEXT,
        symbol TEXT,
        expiration DATE,
        strike REAL,
        type TEXT,
        last REAL,
        mark REAL,
        bid REAL,
        bid_size INTEGER,
        ask REAL,
        ask_size INTEGER,
        volume INTEGER,
        open_interest INTEGER,
        date DATE,
        implied_volatility REAL,
        delta REAL,
        gamma REAL,
        theta REAL,
        vega REAL,
        rho REAL
    );
    '''

    # Execute the create table query
    cursor.execute(create_table_query)
    conn.commit()

    # Close the connection
    cursor.close()
    conn.close()

    print("Table created successfully.")

# Call the function to create the table
create_options_table(database_path)

# Historical data import

In [16]:
def validate_date(date_input):
    # List of expected date formats
    date_formats = ['%Y-%m-%d', '%Y%m%d', '%d%m%Y', '%m%d%Y', '%d-%m-%Y', '%m-%d-%Y']

    # Try parsing the date with expected formats
    for fmt in date_formats:
        try:
            # Parse date
            parsed_date = datetime.datetime.strptime(date_input, fmt)
            formatted_date = parsed_date.strftime('%Y-%m-%d')

            # Check if parsed date is a weekend (Saturday or Sunday)
            if parsed_date.weekday() >= 5:
                return False, "Weekend dates are not allowed. Please re-enter a valid date."

            # If parsing is successful and not a weekend date, return the formatted date
            return True, formatted_date
        except ValueError:
            continue

    # If no format matched and could not parse the date
    return False, "Could not recognize the date format. Please re-enter the date."

def validate_ticker_symbol(symbol):
    ticker_data = yf.Ticker(symbol)
    hist = ticker_data.history(period="5d")
    if hist.empty:
        return False, "Ticker symbol does not exist or no data available."
    else:
        return True, "Ticker symbol is valid."

def check_duplicate_entry(cursor, symbol, date):
    cursor.execute('''
        SELECT COUNT(*) FROM historical_options
        WHERE symbol = ? AND date = ?
    ''', (symbol, date))
    count = cursor.fetchone()[0]
    return count > 0

def get_user_confirmation(formatted_date):
    # Simulating a user interface element for confirmation
    print(f"Is the date {formatted_date} correct?")
    print("[Yes] Confirm and proceed")
    print("[No] Re-enter the date")
    # Here you would capture user input, e.g., via a button click in a GUI or a response in a CLI
    user_input = input("Enter your choice (Yes/No): ").strip().lower()
    return user_input == "yes"

def is_weekend(date):
    # Check if the date is a weekend (Saturday or Sunday)
    return date.weekday() >= 5

def fetch_and_store_options_data(database_path, api_key):
    conn = sqlite3.connect(database_path, isolation_level=None)
    cursor = conn.cursor()

    while True:
        symbol = input("Enter the ticker symbol (e.g., 'DELL'): ").upper()
        is_valid_symbol, message_symbol = validate_ticker_symbol(symbol)
        if not is_valid_symbol:
            print(message_symbol)
            continue
        else:
            break

    while True:
        from_date_input = input("Enter the start date (e.g., '2024-06-14'): ")
        is_valid_from_date, formatted_from_date = validate_date(from_date_input)
        if not is_valid_from_date:
            print(formatted_from_date)
            continue
        else:
            break

    to_date_input = input("Enter the end date (leave blank for a single day): ")
    if to_date_input.strip():
        is_valid_to_date, formatted_to_date = validate_date(to_date_input)
        if not is_valid_to_date:
            print(formatted_to_date)
            return
    else:
        formatted_to_date = formatted_from_date  # Default to from_date if to_date is not provided

    # Check if the date range is too large for API limits
    delta = datetime.datetime.strptime(formatted_to_date, '%Y-%m-%d') - datetime.datetime.strptime(formatted_from_date, '%Y-%m-%d')
    if delta.days > 25:
        print("Date range exceeds API call limit (25 days). Please enter a smaller date range.")
        return

    current_date = datetime.datetime.strptime(formatted_from_date, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(formatted_to_date, '%Y-%m-%d')

    print(f"Fetching data from {formatted_from_date} to {formatted_to_date}...")

    while current_date <= end_date:
        current_date_str = current_date.strftime('%Y-%m-%d')

        if is_weekend(current_date):
            print(f"Skipping weekend date: {current_date_str}")
            current_date += datetime.timedelta(days=1)
            continue

        print(f"Processing data for date: {current_date_str}")

        # Check for duplicate entry
        if check_duplicate_entry(cursor, symbol, current_date_str):
            print(f"Data for symbol '{symbol}' and date '{current_date_str}' already exists in the database.")
        else:
            print(f"Symbol '{symbol}' and date '{current_date_str}' passed duplication checking.")

            # Continue with fetching and storing data...
            # Example URL and params for API request
            url = "https://www.alphavantage.co/query"
            params = {
                "function": "HISTORICAL_OPTIONS",
                "symbol": symbol,
                "date": current_date_str,
                "apikey": api_key
            }

            print(f"Making API request to fetch data for date: {current_date_str}...")
            try:
                response = requests.get(url, params=params)
                data = response.json()
                if 'Note' in data or 'Error Message' in data:
                    print(data.get('Note') or data.get('Error Message'))
                    return
            except requests.exceptions.RequestException as e:
                print(f"An error occurred while making the API request: {e}")
                return

            print(f"Data fetched successfully for date: {current_date_str}. Storing in database...")
            for entry in data.get('data', []):
                cursor.execute('''
                    INSERT INTO historical_options (
                        contractID, symbol, expiration, strike, type, last, mark, bid, bid_size, ask, ask_size,
                        volume, open_interest, date, implied_volatility, delta, gamma, theta, vega, rho
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    entry['contractID'], entry['symbol'], entry['expiration'], entry['strike'], entry['type'],
                    entry['last'], entry['mark'], entry['bid'], entry['bid_size'], entry['ask'], entry['ask_size'],
                    entry['volume'], entry['open_interest'], entry['date'], entry['implied_volatility'],
                    entry['delta'], entry['gamma'], entry['theta'], entry['vega'], entry['rho']
                ))

            conn.commit()
            print(f"Data stored successfully in the database for date: {current_date_str}")

        # Move to the next date
        current_date += datetime.timedelta(days=1)

    cursor.close()
    conn.close()

api_key = 'H43KYGRNGLWUR13H'
fetch_and_store_options_data(database_path, api_key)

Enter the ticker symbol (e.g., 'DELL'): dell
Enter the start date (e.g., '2024-06-14'): 20240606
Enter the end date (leave blank for a single day): 20240612
Fetching data from 2024-06-06 to 2024-06-12...
Processing data for date: 2024-06-06
Symbol 'DELL' and date '2024-06-06' passed duplication checking.
Making API request to fetch data for date: 2024-06-06...
Data fetched successfully for date: 2024-06-06. Storing in database...
Data stored successfully in the database for date: 2024-06-06
Processing data for date: 2024-06-07
Symbol 'DELL' and date '2024-06-07' passed duplication checking.
Making API request to fetch data for date: 2024-06-07...
Data fetched successfully for date: 2024-06-07. Storing in database...
Data stored successfully in the database for date: 2024-06-07
Processing data for date: 2024-06-08
Symbol 'DELL' and date '2024-06-08' passed duplication checking.
Making API request to fetch data for date: 2024-06-08...
Data fetched successfully for date: 2024-06-08. Stori

# Query data

In [17]:
# Inspect data in table
def view_options_data(database_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Query to select all data from the table
    select_query = 'SELECT symbol, date, count(*) FROM historical_options where symbol = "DELL" group by date;'
    # select_query = 'SELECT * FROM historical_options;'
    # select_query = 'SELECT * FROM historical_options where symbol = "DELL" and date = "2024-06-17";'

    # Execute the query
    cursor.execute(select_query)

    # Get column names from cursor description
    columns = [description[0] for description in cursor.description]

    # Print column names
    print(", ".join(columns))

    # Fetch all rows from the database
    rows = cursor.fetchall()

    # Print each row
    for row in rows:
        print(row)

    # Close the connection
    cursor.close()
    conn.close()

view_options_data(database_path)

symbol, date, count(*)
('DELL', '2024-06-05', 1470)
('DELL', '2024-06-06', 1538)
('DELL', '2024-06-07', 1576)
('DELL', '2024-06-10', 1464)
('DELL', '2024-06-11', 1464)
('DELL', '2024-06-12', 1470)
('DELL', '2024-06-13', 1540)
('DELL', '2024-06-14', 1584)
('DELL', '2024-06-17', 1510)
('DELL', '2024-06-18', 3020)


# Duplication Checking

In [11]:
def check_exact_duplicates_in_table(cursor):
    cursor.execute('''
        SELECT contractID, expiration, date, COUNT(*)
        FROM historical_options
        GROUP BY contractID, expiration, date
        HAVING COUNT(*) > 1
    ''')

    duplicates = cursor.fetchall()

    if duplicates:
        print("Duplicates found based on contractID, expiration, and date:")
        for duplicate in duplicates:
            print(f"Contract ID: {duplicate[0]}, Expiration: {duplicate[1]}, Date: {duplicate[2]}, Count: {duplicate[3]}")
    else:
        print("No duplicates found based on contractID, expiration, and date.")

# Example usage assuming 'database_path' is defined
def main_check():
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    check_exact_duplicates_in_table(cursor)

    cursor.close()
    conn.close()

main_check()


No duplicates found based on contractID, expiration, and date.
